<i>Copyright (c) Microsoft Corporation.</i>

<i>Licensed under the MIT License.</i>

# Data Preparation for Retail Sales Forecasting

This notebook introduces how to split the Orange Juice dataset into training sets and test sets for training and evaluating different retail sales forecasting methods.

We use backtesting a method that tests a predictive model on historical data to evaluate the forecasting methods. Other than standard [K-fold cross validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)) which randomly splits data into K folds, we split the data so that any of the time stamps in the training set is no later than any of the time stamps in the test set.



## Global Settings and Imports

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import sys

# import math
# import datetime
import warnings

# import numpy as np
# import pandas as pd
# import tqdm as tqdm
# import scrapbook as sb
# import matplotlib.pyplot as plt

import forecasting_lib.common.forecast_settings as fs
from forecasting_lib.common.utils import git_repo_path
from forecasting_lib.dataset.ojdata import download_ojdata, split_train_test


warnings.filterwarnings("ignore")

print("System version: {}".format(sys.version))

System version: 3.6.7 | packaged by conda-forge | (default, Nov  6 2019, 16:19:42) 
[GCC 7.3.0]


In [7]:
# Use False if you've already downloaded and split the data
DOWNLOAD_DATA = True

# Data directory
DATA_DIR = os.path.join(git_repo_path(), "ojdata")

## Data Preparation

We need to download the Orange Juice data and split it into training and test sets. By default, the following cell will download and spit the data. If you've already done so, you may skip this part by switching `DOWNLOAD_SPLIT_DATA` to `False`.

We store the training data and test data using dataframes. The training data includes `train_df` and `aux_df` with `train_df` containing the historical sales up to week 135 (the time we make forecasts) and `aux_df` containing price/promotion information up until week 138. Here we assume that future price and promotion information up to a certain number of weeks ahead is predetermined and known. The test data is stored in `test_df` which contains the sales of each product in week 137 and 138. Assuming the current week is week 135, our goal is to forecast the sales in week 137 and 138 using the training data. There is a one-week gap between the current week and the first target week of forecasting as we want to leave time for planning inventory in practice. The setting of the forecast problem is defined in `forecasting_lib.common.forecast_settings`. We can change this setting (e.g., modify the horizon of the forecast or the range of the historical data) by updating this setting.

In [5]:
if DOWNLOAD_DATA:
    download_ojdata(DATA_DIR)

Data already exists at the specified location.


In [8]:
data_generator = split_train_test(DATA_DIR, fs)
[train_df, test_df, aux_df] = next(data_generator)
train_df.reset_index(inplace=True)
test_df.reset_index(inplace=True)
aux_df.reset_index(inplace=True)

In [10]:
train_df

,store,brand,week,logmove,constant,price1,price2,price3,price4,price5,price6,price7,price8,price9,price10,price11,deal,feat,profit
0,2,1,40,9.018695,1,0.060469,0.060497,0.042031,0.029531,0.049531,0.053021,0.038906,0.041406,0.028906,0.024844,0.038984,1,0.0,37.992326
1,2,1,46,8.723231,1,0.060469,0.060312,0.045156,0.046719,0.049531,0.047813,0.045781,0.027969,0.042969,0.042031,0.038984,0,0.0,30.126667
2,2,1,47,8.253228,1,0.060469,0.060312,0.045156,0.046719,0.037344,0.053021,0.045781,0.041406,0.048125,0.032656,0.038984,0,0.0,30.000000
3,2,1,48,8.987197,1,0.060469,0.060312,0.049844,0.037344,0.049531,0.053021,0.045781,0.041406,0.042344,0.032656,0.038984,0,0.0,29.950000
4,2,1,50,9.093357,1,0.060469,0.060312,0.043594,0.031094,0.049531,0.053021,0.046648,0.041406,0.042344,0.032656,0.038203,0,0.0,29.920000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84178,137,11,131,9.631154,1,0.027969,0.051979,0.049080,0.039820,0.031094,0.048395,0.037500,0.038906,0.023281,0.022187,0.025703,1,0.0,17.170000
84179,137,11,132,9.704061,1,0.030504,0.051979,0.043594,0.033927,0.033167,0.045729,0.031094,0.038906,0.025313,0.024844,0.026328,1,1.0,18.630000
84180,137,11,133,8.995165,1,0.043056,0.051979,0.045542,0.031094,0.037205,0.046579,0.033470,0.037969,0.020156,0.025625,0.029609,1,0.0,25.350000
84181,137,11,134,8.912473,1,0.039062,0.049301,0.049588,0.032300,0.031094,0.050937,0.042031,0.035781,0.022031,0.031094,0.029609,1,0.0,25.320000


## Split Data for Multi-Round Forecasting

We can also create training data and test data for multi-round forecasting. In this case, we gradually increase the length of the training data at each round. This allows us to retrain the forecasting model for achieving more accurate forecasts. Using the default settings in `forecasting_lib/common/forecast_settings.py` and update `NUM_ROUNDS` to 12, we can generate the training and test sets as follows


| **Round** | **Train period <br> start week** | **Train period <br> end week** | **Test period <br> start week** | **Test period <br> end week** |
| -------- | --------------- | ------------------ | ------------------------- | ----------------------- |
| 1 | 40 | 135 | 137 | 138 |
| 2 | 40 | 137 | 139 | 140 |
| 3 | 40 | 139 | 141 | 142 |
| 4 | 40 | 141 | 143 | 144 |
| 5 | 40 | 143 | 145 | 146 |
| 6 | 40 | 145 | 147 | 148 |
| 7 | 40 | 147 | 149 | 150 |
| 8 | 40 | 149 | 151 | 152 |
| 9 | 40 | 151 | 153 | 154 |
| 10 | 40 | 153 | 155 | 156 |
| 11 | 40 | 155 | 157 | 158 |
| 12 | 40 | 157 | 159 | 160 |

The gap of one week between training period and test period allows store managers to prepare the stock based on the forecasted demand. Besides, we assume that the information about the price, deal, and advertisement up until the forecast period end week is available at each round.

In [11]:
fs.NUM_ROUNDS = 12
for train_df, test_df, aux_df in split_train_test(DATA_DIR, fs):
    print("Training data size: {}".format(train_df.shape))
    print("Testing data size: {}".format(test_df.shape))
    print("Auxiliary data size: {}".format(aux_df.shape))
    print("Minimum training week number: {}".format(min(train_df["week"])))
    print("Maximum training week number: {}".format(max(train_df["week"])))
    print("Minimum testing week number: {}".format(min(test_df["week"])))
    print("Maximum testing week number: {}".format(max(test_df["week"])))
    print("Minimum auxiliary week number: {}".format(min(aux_df["week"])))
    print("Maximum auxiliary week number: {}".format(max(aux_df["week"])))
    print("")

Training data size: (84183, 18)
Testing data size: (1826, 18)
Auxiliary data size: (86911, 15)
Minimum training week number: 40
Maximum training week number: 135
Minimum testing week number: 137
Maximum testing week number: 138
Minimum auxiliary week number: 40
Maximum auxiliary week number: 138

Training data size: (85998, 18)
Testing data size: (1793, 18)
Auxiliary data size: (88704, 15)
Minimum training week number: 40
Maximum training week number: 137
Minimum testing week number: 139
Maximum testing week number: 140
Minimum auxiliary week number: 40
Maximum auxiliary week number: 140

Training data size: (87802, 18)
Testing data size: (1771, 18)
Auxiliary data size: (90475, 15)
Minimum training week number: 40
Maximum training week number: 139
Minimum testing week number: 141
Maximum testing week number: 142
Minimum auxiliary week number: 40
Maximum auxiliary week number: 142

Training data size: (89617, 18)
Testing data size: (1749, 18)
Auxiliary data size: (92224, 15)
Minimum tra

## Additional Reading

\[1\] Christoph GBergmeir, Rob J. Hyndman, and Bonsoo Koo. 2018. A Note on the Validity of Cross-Validation for Evaluating Autoregressive Time Series Prediction. Computational Statistics & Data Analysis. 120, pp. 70-83.<br>
\[2\] How To Backtest Machine Learning Models for Time Series Forecasting: https://machinelearningmastery.com/backtest-machine-learning-models-time-series-forecasting/Parameters.rst <br>

